**Run with conda env: allele_specific**

### Preparing the data for TxBurst inference 

In [1]:
cd /staging/leuven/stg_00041/Adrian/TALON_JANISZEWSKI_XCR2/

/lustre1/project/stg_00041/Adrian/TALON_JANISZEWSKI_XCR2


In [2]:
import pandas as pd
import numpy as np
import sys

## Calculate RPKMs manually with numpy
#### Import raw allelic count table 



In [3]:
allelic_mtx = pd.read_csv("allele_specific/pre_processing/AJ_0065.9_allelic_count_mtx.csv").sort_values(["GENE"]).reset_index()

/data/leuven/320/vsc32023/miniconda/envs/allele_specific/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
allelic_anno = pd.read_csv("allele_specific/txburst/genes.sorted.bed", sep = "\t", names=["chr","start","end","GENE"])

In [5]:
allelic_anno_match = allelic_anno[allelic_anno["GENE"] \
                                  .isin(allelic_mtx["GENE"])] \
                                  .drop_duplicates(subset=["GENE"]) \
                                  .sort_values(["GENE"]) \
                                  .reset_index()

In [6]:
# Assert matching between expr matrix and gene anno and extract matching index


pd.testing.assert_series_equal(allelic_mtx['GENE'], allelic_anno_match['GENE'])
allelic_mtx = allelic_mtx.set_index("GENE")
allelic_anno_match = allelic_anno_match.set_index("GENE")

matched_index = pd.Index.intersection(allelic_mtx.index, allelic_anno_match.index)

In [7]:
# Calculate gene lengths

allelic_anno_match["gene_len"] = allelic_anno_match["end"] - allelic_anno_match["start"]

In [8]:
# Clean matrix

allelic_mtx = allelic_mtx.drop(["index","Unnamed: 0","CHROM"], axis = 1)

In [9]:
# Split expr matrix by the allele
allelic_mtx_129 = allelic_mtx.filter(like=".129")
allelic_mtx_129.columns = allelic_mtx_129.columns.str.replace("\\.129","")

allelic_mtx_Cast = allelic_mtx.filter(like=".Cast")
allelic_mtx_Cast.columns = allelic_mtx_Cast.columns.str.replace("\\.Cast","")

In [10]:
counts = np.asarray(allelic_mtx.loc[matched_index], dtype=int)

counts_129 = np.asarray(allelic_mtx_129.loc[matched_index], dtype=int)
counts_Cast = np.asarray(allelic_mtx_Cast.loc[matched_index], dtype=int)

gene_lengths = np.asarray(allelic_anno_match.loc[matched_index]['gene_len'],
                          dtype=int)

gene_names = np.array(matched_index)

In [11]:
print(counts.shape)
print(counts_129.shape)
print(counts_Cast.shape)
print(gene_lengths.shape)

(15912, 1324)
(15912, 662)
(15912, 662)
(15912,)


In [12]:
# Function copied from Elegant NumPy book
# https://www.oreilly.com/library/view/elegant-scipy/9781491922927/ch01.html

def rpkm(counts, lengths):
    """Calculate reads per kilobase transcript per million reads.

    RPKM = (10^9 * C) / (N * L)

    Where:
    C = Number of reads mapped to a gene
    N = Total mapped reads in the experiment
    L = Exon length in base pairs for a gene

    Parameters
    ----------
    counts: array, shape (N_genes, N_samples)
        RNAseq (or similar) count data where columns are individual samples
        and rows are genes.
    lengths: array, shape (N_genes,)
        Gene lengths in base pairs in the same order
        as the rows in counts.

    Returns
    -------
    normed : array, shape (N_genes, N_samples)
        The RPKM normalized counts matrix.
    """
    N = np.sum(counts, axis=0)  # sum each column to get total reads per sample
    L = lengths
    C = counts

    normed = 1e9 * C / (N[np.newaxis, :] * L[:, np.newaxis])

    return(normed)

In [13]:
allelic_rpkm = rpkm(counts, gene_lengths)

allelic_129_rpkm = rpkm(counts_129, gene_lengths)
allelic_Cast_rpkm = rpkm(counts_Cast, gene_lengths)

/data/leuven/320/vsc32023/miniconda/envs/allele_specific/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in true_divide
/data/leuven/320/vsc32023/miniconda/envs/allele_specific/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide


In [14]:
allelic_rpkm_df = pd.DataFrame(data=allelic_rpkm, index=matched_index, columns=allelic_mtx.columns)

allelic_129_rpkm_df = pd.DataFrame(data=allelic_129_rpkm, index=matched_index, columns=allelic_mtx_129.columns)
allelic_Cast_rpkm_df = pd.DataFrame(data=allelic_Cast_rpkm, index=matched_index, columns=allelic_mtx_Cast.columns)

In [15]:
allelic_rpkm_df.to_csv("allele_specific/txburst/allelic_mtx_rpkm.csv")

allelic_129_rpkm_df.to_csv("allele_specific/txburst/allelic_mtx_129_rpkm.csv")
allelic_Cast_rpkm_df.to_csv("allele_specific/txburst/allelic_mtx_Cast_rpkm.csv")

### Distinguish missing data from no-expression

From Larsson TxBursts github:  
    “If you have estimated the allelic transcript counts from the fraction of allelic reads,  
    it is important to consider what is missing data in this case. Genes with expression (reads)  
    but no allelic reads are different from genes without expression (and therefore no allelic reads).  
    We handle the first case as missing data, since it is not possible to assign the expression.  
    In the second case, we replace the NaN with a 0 since there is genuinely no detected expression.  
    Omitting this step may have severe effects on the quality of the inference.”

In [16]:
# Import non allelic count matrix
counts_nonAllelic = pd.read_csv("allele_specific/txburst/AJ_0065_seurat_normCounts.csv", 
                               index_col=0)

In [17]:
# Convert tables to a long format

allelic_129_rpkm_long = allelic_129_rpkm_df \
    .reset_index() \
    .melt(id_vars = "GENE", var_name = "cell", value_name="129_rpkm") 

allelic_Cast_rpkm_long = allelic_Cast_rpkm_df \
    .reset_index() \
    .melt(id_vars = "GENE", var_name = "cell", value_name="Cast_rpkm") 

counts_nonAllelic_long = counts_nonAllelic \
    .rename_axis("GENE") \
    .reset_index() \
    .melt(id_vars = "GENE", var_name = "cell", value_name = "nonAllelic") 

In [18]:
# Merge the allelic tables and calculate sumReads of the two alleles and allelic ratio (129/sumReads)

pd.testing.assert_series_equal(allelic_129_rpkm_long["GENE"],allelic_Cast_rpkm_long["GENE"])
pd.testing.assert_series_equal(allelic_129_rpkm_long["cell"],allelic_Cast_rpkm_long["cell"])

allelic_129_Cast_rpkm = allelic_129_rpkm_long \
                                .merge(
                                allelic_Cast_rpkm_long,
                                on=["GENE","cell"], how="left",
                                validate = "one_to_one") 

allelic_129_Cast_rpkm["sumReads"] = allelic_129_Cast_rpkm["129_rpkm"] + allelic_129_Cast_rpkm["Cast_rpkm"]

In [19]:
# Filter join with smaller nonAllelic table which is after seurat cell/gene filtering

counts_all = counts_nonAllelic_long \
                .merge(allelic_129_Cast_rpkm,
                      on=["GENE","cell"], how = "left")

In [128]:
# Handle missing data and no expression

## Genes with expression (reads, nonAllelic > 0) 
## but 
## no allelic reads (sumReads 0 or NaN) => NaN

# are different from 

## genes without expression (nonAllelic = 0) 
## and therefore 
## no allelic reads (sumReads = 0 or NaN) => 0


def data_handler(df, allele):
    
    """
    df is the name of the dataframe
    allele is the name of the column which will be corrected, either 129 or Cast
    """
    
    if df['nonAllelic'] > 0 and (df['sumReads'] == 0 or np.isnan(df['sumReads'])):
                               return float('NaN')
    if df['nonAllelic'] == 0 and (df['sumReads'] == 0 or np.isnan(df['sumReads'])):
                               return 0
    return df[allele]

In [141]:
# This is not time efficient and will take long
# For optimization vectorize with np.select

counts_all['129_rpkm_corr'] = counts_all.apply(data_handler, allele = '129_rpkm', axis=1)
counts_all['Cast_rpkm_corr'] = counts_all.apply(data_handler, allele = 'Cast_rpkm', axis=1)

In [365]:
counts_all.to_csv("allele_specific/txburst/allelic_counts_corrected.csv")

In [335]:
# Remove mitochondrial genes and GFP

counts_all_filt = counts_all[~counts_all["GENE"].str.contains('mt\\-|pCX-eGFP', na=False)]

# Remove Day 12 with Cast reactivated

counts_all_filt = counts_all_filt[~counts_all_filt["cell"].str.contains('Day_12_Xi_Cast', na=False)]

In [343]:
# Merge with information about clusters and pseuodotime

cell_cluster_pseudo = pd.read_csv('allele_specific/txburst/seurat_data_pseudotime_clusters.csv')
cell_cluster_pseudo = cell_cluster_pseudo[["Name","Pseudotime","State","seurat_clusters_rename"]].rename(columns={"Name":"cell", "seurat_clusters_rename":"cluster"})

counts_all_filt = counts_all_filt \
                                .merge(
                                cell_cluster_pseudo,
                                on=["cell"], how="left",
                                validate = "many_to_one") 

In [366]:
counts_all_filt.to_csv("allele_specific/txburst/allelic_counts_all_filt.csv")

In [362]:
def long_to_mtx(df_long,
                allele,
                timepoint=None, 
                cluster=None):
    """ Convert long format gene expression back to genes x cells matrix for each 
    cell type or cluster for each allele. 
    
    Generates a valid input for txburstML.py
    
    timepoint: string
       A strings that will be used to grep timepoints from "cell" column
    cluster: string
        A string that selects the cluster from cluster column
    allele: string
       String that will indicate which allele to choose. Here, either: 'Cast_rpkm_corr'
           or '129_rpkm_corr'
    """
    if timepoint:   
        mtx = df_long[df_long['cell'].str.contains(timepoint)][['GENE','cell',allele]] \
            .pivot(index='GENE',columns='cell',values=allele)
    
        mtx.to_csv("allele_specific/txburst/{}_{}.csv".format(timepoint, allele))  
    else:  
        mtx = df_long[df_long['cluster']==cluster][['GENE','cell',allele]] \
            .pivot(index='GENE',columns='cell',values=allele)
    
        mtx.to_csv("allele_specific/txburst/clust_{}_{}.csv".format(cluster, allele))

In [364]:
timepoints = ['Day_0','Day_8','Day_9','Day_10','Day_12','iPSC']
alleles = ['129_rpkm_corr', 'Cast_rpkm_corr']

for day in timepoints:
    for allele in alleles:
        long_to_mtx(counts_all_filt, allele = allele, timepoint = day)

In [363]:
clusters = [0,1,2,3,4,5]
alleles = ['129_rpkm_corr', 'Cast_rpkm_corr']

for cluster in clusters:
    for allele in alleles:
        long_to_mtx(counts_all_filt, allele = allele, cluster = cluster)

In [291]:
# Prepare the table with selected sample and allele
# For starters Xa in Day 0 vs Xa in iPSCs

#D0_Xa = counts_all_filt[counts_all_filt['cell'].str.contains("Day_0")][['GENE','cell','Cast_rpkm_corr']] \
#            .pivot(index='GENE',columns='cell',values='Cast_rpkm_corr')

#D12_Xa = counts_all_filt[counts_all_filt['cell'].str.contains("Day_12_Xi_Mus")][['GENE','cell','Cast_rpkm_corr']] \
#            .pivot(index='GENE',columns='cell',values='Cast_rpkm_corr')

#D0_Xa.to_csv("data/D0_Xa_rpkm.csv")
#D12_Xa.to_csv("data/D12_Xa_rpkm.csv")